Для произвольного набора данных решить задачу:
1. Классификации
  - случ. лес;
  - логистическая регрессия
  - градиентный бустинг (использовать пакет XGBoost)
2. Регрессии
   - случ. лес;
   - линейн. регрессия;
   - градиент. бустинг
 

# Классификация

In [ ]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pylab as plt
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression

import xgboost

In [ ]:
# https://www.kaggle.com/mnassrib/telecom-churn-datasets
df = pd.read_csv('telecom_churn.csv')

Название;Описание;Тип
* State;Буквенный код штата;номинальный
* Account length;Как долго клиент обслуживается компанией;количественный
* Area code;Префикс номера телефона;количественный
* International plan;Международный роуминг (подключен/не подключен);бинарный
* Voice mail plan;Голосовая почта (подключена/не подключена);бинарный
* Number vmail messages;Количество голосовых сообщений;количественный
* Total day minutes;Общая длительность разговоров днем;количественный
* Total day calls;Общее количество звонков днем;количественный
* Total day charge;Общая сумма оплаты за услуги днем;количественный
* Total eve minutes;Общая длительность разговоров вечером;количественный
* Total eve calls;Общее количество звонков вечером;количественный
* Total eve charge;Общая сумма оплаты за услуги вечером;количественный
* Total night minutes;Общая длительность разговоров ночью;количественный
* Total night calls;Общее количество звонков ночью;количественный
* Total night charge;Общая сумма оплаты за услуги ночью;количественный
* Total intl minutes;Общая длительность международных разговоров;количественный
* Total intl calls;Общее количество международных разговоров;количественный
* Total intl charge;Общая сумма оплаты за международные разговоры;количественный
* Customer service calls;Число обращений в сервисный центр;количественный
* Churn; Отток клиентов; Бинарный 

In [ ]:
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [ ]:
df.drop(['state', 'phone number'], axis=1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   account length          3333 non-null   int64  
 1   area code               3333 non-null   int64  
 2   international plan      3333 non-null   object 
 3   voice mail plan         3333 non-null   object 
 4   number vmail messages   3333 non-null   int64  
 5   total day minutes       3333 non-null   float64
 6   total day calls         3333 non-null   int64  
 7   total day charge        3333 non-null   float64
 8   total eve minutes       3333 non-null   float64
 9   total eve calls         3333 non-null   int64  
 10  total eve charge        3333 non-null   float64
 11  total night minutes     3333 non-null   float64
 12  total night calls       3333 non-null   int64  
 13  total night charge      3333 non-null   float64
 14  total intl minutes      3333 non-null   

In [ ]:
# Выделим целевую переменную. Изменим ее тип
df['churn'] = df['churn'].astype('int')

In [ ]:
# Переведем некоторые признаки в бинарные 
to_binar = {'yes': 1, 'no': 0}
df['international plan'] = df['international plan'].map(to_binar)
df['voice mail plan'] = df['voice mail plan'].map(to_binar)

In [ ]:
df.head()

,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [ ]:
# Признаки
X = df.drop('churn', axis=1)
X

,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,0,1,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2
3329,68,415,0,0,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3
3330,28,510,0,0,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2
3331,184,510,1,0,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2


In [ ]:
# Целевая переменная
y = df['churn']
y

0       0
1       0
2       0
3       0
4       0
       ..
3328    0
3329    0
3330    0
3331    0
3332    0
Name: churn, Length: 3333, dtype: int64

In [ ]:
# Делим выборку на обучающую и тестовую
X_train, X_holdout, y_train, y_holdout = train_test_split(X.values, y, test_size=0.3, random_state=17)

### Случайный лес

In [ ]:
# Объявим модель
rfc = RandomForestClassifier(random_state=17)

In [ ]:
# Обучим модель
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [ ]:
# Оценим модель
rfc_pred = rfc.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'RandomForestClassifier'
pred_result.loc[0,'accuracy_score'] = accuracy_score(y_holdout, rfc_pred)

In [ ]:
# Вывод рез-та 
pred_result

,model,accuracy_score
0,RandomForestClassifier,0.951


### Логистическая регрессия

In [ ]:
# Объявим модель
lr = LogisticRegression(random_state=17)

In [ ]:
# Обучим модель
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# Оценим модель
lr_pred = lr.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'LogisticRegression'
pred_result.loc[0,'accuracy_score'] = accuracy_score(y_holdout, lr_pred)

In [ ]:
# Вывод рез-та 
pred_result

,model,accuracy_score
0,LogisticRegression,0.872


### Градиентный бустинг

In [ ]:
# Объявим модель
xgbcl = xgboost.XGBClassifier(random_state=17)

In [ ]:
# Обучим модель
xgbcl.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=17,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
# Оценим модель
xgbcl_pred = xgbcl.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'XGBClassifier'
pred_result.loc[0,'accuracy_score'] = accuracy_score(y_holdout, xgbcl_pred)

In [ ]:
# Вывод рез-та 
pred_result

,model,accuracy_score
0,XGBClassifier,0.951


# Регрессия 

In [ ]:
# https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset 
df = pd.read_csv('day.csv',sep = ',')


- instant: record index 
- dteday : date 
- season : season (1:winter, 2:spring, 3:summer, 4:fall) 
- yr : year (0: 2011, 1:2012) 
- mnth : month ( 1 to 12) 
- hr : hour (0 to 23) 
- holiday : weather day is holiday or not (extracted from [Web Link]) 
- weekday : day of the week 
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0. 
+ weathersit : 
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale) 
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale) 
- hum: Normalized humidity. The values are divided to 100 (max) 
- windspeed: Normalized wind speed. The values are divided to 67 (max) 
- casual: count of casual users 
- registered: count of registered users 
- cnt: count of total rental bikes including both casual and registered 



In [ ]:
df.head(3)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0.0,1.0,0.0,6.0,0.0,2.0,0.344167,0.363625,0.805833,0.160446,331.0,654.0,985.0
1,2,2011-01-02,1,0.0,1.0,0.0,0.0,0.0,2.0,0.363478,0.353739,0.696087,0.248539,131.0,670.0,801.0
2,3,2011-01-03,1,0.0,1.0,0.0,1.0,1.0,1.0,0.196364,0.189405,0.437273,0.248309,120.0,1229.0,1349.0


In [ ]:
# Удалим строки с пустыми значениями
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 747
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     744 non-null    object 
 1   dteday      744 non-null    object 
 2   season      744 non-null    object 
 3   yr          744 non-null    float64
 4   mnth        744 non-null    float64
 5   holiday     744 non-null    float64
 6   weekday     744 non-null    float64
 7   workingday  744 non-null    float64
 8   weathersit  744 non-null    float64
 9   temp        744 non-null    float64
 10  atemp       744 non-null    float64
 11  hum         744 non-null    float64
 12  windspeed   744 non-null    float64
 13  casual      744 non-null    float64
 14  registered  744 non-null    float64
 15  cnt         744 non-null    float64
dtypes: float64(13), object(3)
memory usage: 98.8+ KB


In [ ]:
df.describe()

,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000
mean,0.491935,6.423387,0.028226,2.997312,0.689516,1.388441,0.490695,0.470072,0.624558,0.190069,834.509409,3603.924731,4437.280914
std,0.500271,3.497285,0.165729,1.987171,0.463003,0.542588,0.184824,0.164695,0.143380,0.076881,688.272509,1595.466192,1985.043939
min,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,0.000000,3.000000,0.000000,1.000000,0.000000,1.000000,0.329167,0.326382,0.512500,0.134954,302.250000,2447.500000,3041.000000
50%,0.000000,6.000000,0.000000,3.000000,1.000000,1.000000,0.488333,0.476011,0.623125,0.178802,697.000000,3630.000000,4499.500000
75%,1.000000,9.000000,0.000000,5.000000,1.000000,2.000000,0.653541,0.607962,0.729583,0.232689,1082.750000,4740.750000,5908.250000
max,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [ ]:
df.corr()[abs(df.corr()) >=0.6]

,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
yr,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.603292,NaN
mnth,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
holiday,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workingday,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weathersit,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
temp,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.991999,NaN,NaN,NaN,NaN,0.644163
atemp,NaN,NaN,NaN,NaN,NaN,NaN,0.991999,1.000000,NaN,NaN,NaN,NaN,0.648182
hum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
windspeed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [ ]:
# удалим 'temp', тк он коррелирует с признаком "atemp", но 'temp' больше влияет на таргет "cnt"
df.drop('temp', axis=1, inplace=True)

In [ ]:
# удалим 'instant', тк он уникальный для всех
df.drop('instant', axis=1, inplace=True)

In [ ]:
# удалим столбец, т.к не можем строить прогноз по его значениям
df.drop('dteday', axis=1, inplace=True)

In [ ]:
# удаляем registered, исходя из корреляции: итак понятно, что cnt тем выше, чем выше registered
df.drop('registered', axis=1, inplace=True)
df.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,casual,cnt
0,1,0.0,1.0,0.0,6.0,0.0,2.0,0.363625,0.805833,0.160446,331.0,985.0
1,1,0.0,1.0,0.0,0.0,0.0,2.0,0.353739,0.696087,0.248539,131.0,801.0
2,1,0.0,1.0,0.0,1.0,1.0,1.0,0.189405,0.437273,0.248309,120.0,1349.0
3,1,0.0,1.0,0.0,2.0,1.0,1.0,0.212122,0.590435,0.160296,108.0,1562.0
4,1,0.0,1.0,0.0,3.0,1.0,1.0,0.229270,0.436957,0.186900,82.0,1600.0


In [ ]:
df.describe()

,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,casual,cnt
count,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000
mean,0.491935,6.423387,0.028226,2.997312,0.689516,1.388441,0.470072,0.624558,0.190069,834.509409,4437.280914
std,0.500271,3.497285,0.165729,1.987171,0.463003,0.542588,0.164695,0.143380,0.076881,688.272509,1985.043939
min,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.079070,0.000000,0.022392,2.000000,22.000000
25%,0.000000,3.000000,0.000000,1.000000,0.000000,1.000000,0.326382,0.512500,0.134954,302.250000,3041.000000
50%,0.000000,6.000000,0.000000,3.000000,1.000000,1.000000,0.476011,0.623125,0.178802,697.000000,4499.500000
75%,1.000000,9.000000,0.000000,5.000000,1.000000,2.000000,0.607962,0.729583,0.232689,1082.750000,5908.250000
max,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.840896,0.972500,0.507463,3410.000000,8714.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 747
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      744 non-null    object 
 1   yr          744 non-null    float64
 2   mnth        744 non-null    float64
 3   holiday     744 non-null    float64
 4   weekday     744 non-null    float64
 5   workingday  744 non-null    float64
 6   weathersit  744 non-null    float64
 7   atemp       744 non-null    float64
 8   hum         744 non-null    float64
 9   windspeed   744 non-null    float64
 10  casual      744 non-null    float64
 11  cnt         744 non-null    float64
dtypes: float64(11), object(1)
memory usage: 75.6+ KB


In [ ]:
# Доопределим тип для целочисленных столбцов
col = list(df.columns)
col.remove('atemp')
col.remove('hum')
col.remove('windspeed')
col

['season',
 'yr',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'casual',
 'cnt']

In [ ]:
# Информация по фичам (название, кол-во пропущенных значений, число унивкальных значений, питонячий тип)
pd.DataFrame({'Feature': df.columns.tolist(),
              'NaN_count': df.isna().sum().tolist(),
              'number_of_unique_elements': pd.Series(df.columns.tolist()).apply(lambda row: df[row].nunique()),
              'type': pd.Series(df.columns.tolist()).apply(lambda row: df[row].dtype)})

,Feature,NaN_count,number_of_unique_elements,type
0,season,0,5,object
1,yr,0,2,float64
2,mnth,0,12,float64
3,holiday,0,2,float64
4,weekday,0,7,float64
5,workingday,0,2,float64
6,weathersit,0,3,float64
7,atemp,0,690,float64
8,hum,0,595,float64
9,windspeed,0,651,float64


Проверим уникальные значения на предмет ошибок в данных

In [ ]:
df['season'].unique()

array(['1', '2', '3', '4', '[1]'], dtype=object)

Нужно удалить значение "[1]"

In [ ]:
df['yr'].unique()

array([0., 1.])

In [ ]:
df['mnth'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [ ]:
df['holiday'].unique()

array([0., 1.])

In [ ]:
df['weekday'].unique()

array([6., 0., 1., 2., 3., 4., 5.])

In [ ]:
df['workingday'].unique()

array([0., 1.])

In [ ]:
# Удалим строки с неправильным значением признака 'season'
df = df[df['season'] !='[1]']

In [ ]:
for i in col: 
    print(i)
    df[i]= df[i].astype('int')

season
yr
mnth
holiday
weekday
workingday
weathersit
casual
cnt


In [ ]:
# Признаки
X = df.drop('cnt', axis=1)
X

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,casual
0,1,0,1,0,6,0,2,0.363625,0.805833,0.160446,331
1,1,0,1,0,0,0,2,0.353739,0.696087,0.248539,131
2,1,0,1,0,1,1,1,0.189405,0.437273,0.248309,120
3,1,0,1,0,2,1,1,0.212122,0.590435,0.160296,108
4,1,0,1,0,3,1,1,0.229270,0.436957,0.186900,82
...,...,...,...,...,...,...,...,...,...,...,...
743,1,0,1,0,3,1,1,0.229270,0.436957,0.166600,66
744,1,0,1,0,3,1,1,0.229270,0.436957,0.166600,66
745,1,0,1,0,3,1,1,0.229270,0.436957,0.166600,66
746,1,0,1,0,3,1,1,0.229270,0.436957,0.166600,66


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 747
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      742 non-null    int64  
 1   yr          742 non-null    int64  
 2   mnth        742 non-null    int64  
 3   holiday     742 non-null    int64  
 4   weekday     742 non-null    int64  
 5   workingday  742 non-null    int64  
 6   weathersit  742 non-null    int64  
 7   atemp       742 non-null    float64
 8   hum         742 non-null    float64
 9   windspeed   742 non-null    float64
 10  casual      742 non-null    int64  
dtypes: float64(3), int64(8)
memory usage: 69.6 KB


In [ ]:
# Целевая переменная
y = df['cnt']
y

0       985
1       801
2      1349
3      1562
4      1600
       ... 
743     666
744     666
745     666
746     666
747     666
Name: cnt, Length: 742, dtype: int64

Из-за признака "casual" нужно выполнить стандартизацию, тк его значения на порядке выше значений других признаков

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 747
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      742 non-null    int64  
 1   yr          742 non-null    int64  
 2   mnth        742 non-null    int64  
 3   holiday     742 non-null    int64  
 4   weekday     742 non-null    int64  
 5   workingday  742 non-null    int64  
 6   weathersit  742 non-null    int64  
 7   atemp       742 non-null    float64
 8   hum         742 non-null    float64
 9   windspeed   742 non-null    float64
 10  casual      742 non-null    int64  
dtypes: float64(3), int64(8)
memory usage: 69.6 KB


In [ ]:
# Стандартизация
scaler = StandardScaler()
scaler.fit(X)
X_sc = pd.DataFrame(scaler.transform(X), columns=X.columns.copy())

In [ ]:
# Делим выборку на обучающую и тестовую
X_train, X_holdout, y_train, y_holdout = train_test_split(X_sc.values, y, test_size=0.3, random_state=17)

### Случайный лес

In [ ]:
# Объявим модель
rfr = RandomForestRegressor(random_state=17)

In [ ]:
# Обучим модель
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

In [ ]:
# Оценим модель
rfr_pred = rfr.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'RandomForestRegressor'
pred_result.loc[0,'RMSE'] = np.sqrt(mean_squared_error(y_holdout,rfr_pred))

In [ ]:
# Вывод рез-та 
pred_result

,model,RMSE
0,RandomForestRegressor,527.199003


### Линейная регрессия

In [ ]:
# Объявим модель
linr = LinearRegression()

In [ ]:
# Обучим модель
linr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Оценим модель
linr_pred = linr.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'LinearRegression'
pred_result.loc[0,'RMSE'] = np.sqrt(mean_squared_error(y_holdout,linr_pred))

In [ ]:
# Вывод рез-та 
pred_result

,model,RMSE
0,LinearRegression,681.962316


### Градиентный бустинг

In [ ]:
# Объявим модель
gbr = GradientBoostingRegressor(random_state=17)

In [ ]:
# Обучим модель
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=17, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
# Оценим модель
gbr_pred = xgbr.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'GBRegressor'
pred_result.loc[0,'RMSE'] = np.sqrt(mean_squared_error(y_holdout,gbr_pred))

In [ ]:
# Вывод рез-та 
pred_result

,model,RMSE
0,GBRegressor,449.810594


In [ ]:
# Объявим модель
xgbr = xgboost.XGBRegressor(random_state=17)

In [ ]:
# Обучим модель
xgbr.fit(X_train, y_train)

[17:22:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=17,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
# Оценим модель
xgbr_pred = xgbr.predict(X_holdout)
pred_result=pd.DataFrame()
pred_result.loc[0,'model'] = 'XGBRegressor'
pred_result.loc[0,'RMSE'] = np.sqrt(mean_squared_error(y_holdout,xgbr_pred))

In [ ]:
# Вывод рез-та 
pred_result

,model,RMSE
0,XGBRegressor,449.810594


# Вывод


Алгоритм градиентного бустинга дал самый лучший рез-т.